## Split each CIF in the current directory into subcells, keeping only one specified piece of the grid. Outputs: sub_x{ix}_y{iy}_z{iz}_<origname>.cif

In [9]:
import os
import numpy as np
from pymatgen.core import Structure, Lattice

# ─── USER PARAMETERS ──────────────────────────────────────────────────────────

# How many pieces to cut each lattice vector into:
xdiv, ydiv, zdiv = 2, 2, 2

# Which subcell to keep (0‑based indices):
#   0 <= ix < xdiv, 0 <= iy < ydiv, 0 <= iz < zdiv
ix, iy, iz = 0, 0, 0

# ─── END USER PARAMETERS ──────────────────────────────────────────────────────

def extract_subcell(structure, xdiv, ydiv, zdiv, ix, iy, iz):
    # fractional coords array shape (N,3)
    fracs = np.array(structure.frac_coords)

    # Boolean mask of points inside the [ix/xdiv, (ix+1)/xdiv) window, etc.
    mask = (
        (fracs[:, 0] >= ix / xdiv) & (fracs[:, 0] < (ix + 1) / xdiv) &
        (fracs[:, 1] >= iy / ydiv) & (fracs[:, 1] < (iy + 1) / ydiv) &
        (fracs[:, 2] >= iz / zdiv) & (fracs[:, 2] < (iz + 1) / zdiv)
    )
    if not mask.any():
        return None

    # Pull out the subset of species and fractional coords
    species = [structure[i].specie for i, keep in enumerate(mask) if keep]
    sub_fracs = fracs[mask]

    # Shift & re‑normalize fractional coords into [0,1]^3 for the sub‑lattice
    #   new_frac = old_frac * div - index
    scale = np.array([xdiv, ydiv, zdiv])
    index = np.array([ix, iy, iz])
    new_fracs = sub_fracs * scale - index

    # Build the new (smaller) lattice
    old_lat = np.array(structure.lattice.matrix)
    new_lat = old_lat / scale.reshape(3, 1)

    sub_lattice = Lattice(new_lat)
    return Structure(sub_lattice, species, new_fracs)

if __name__ == "__main__":
    for fname in os.listdir("."):
        if not fname.lower().endswith(".cif"):
            continue

        print(f"Reading {fname}…")
        struct = Structure.from_file(fname)
        sub = extract_subcell(struct, xdiv, ydiv, zdiv, ix, iy, iz)
        if sub is None:
            print(f"  → No atoms found in subcell {ix,iy,iz}, skipping.")
            continue

        out_name = f"sub_x{xdiv}_y{ydiv}_z{zdiv}_{fname}"
        print(f"  → Writing {out_name}")
        sub.to(filename=out_name)


Reading ptmsp.cif…
  → Writing sub_x2_y2_z2_ptmsp.cif
Reading PIM1.cif…
  → Writing sub_x2_y2_z2_PIM1.cif
Reading 6fdadurene.cif…
  → Writing sub_x2_y2_z2_6fdadurene.cif


## Make a slice of a CIF to keep only the region within specified X×Y×Z limits. 

In [11]:
import os
import numpy as np
from pymatgen.core import Structure, Lattice

# ─── USER PARAMETERS ──────────────────────────────────────────────────────────

# 1) Define your Cartesian cut‑offs in Å:
XMAX, YMAX, ZMAX = 30.0, 30.0, 30.0

# ─── END USER PARAMETERS ──────────────────────────────────────────────────────

def extract_slice(struct: Structure, x_max, y_max, z_max):
    # cell lengths in Å
    a, b, c = struct.lattice.abc

    # get fractional coords array
    fracs = np.array(struct.frac_coords)    # shape (N,3)

    # convert to Cartesian (orthogonal cell assumed)
    cartesians = np.empty_like(fracs)
    cartesians[:,0] = fracs[:,0] * a
    cartesians[:,1] = fracs[:,1] * b
    cartesians[:,2] = fracs[:,2] * c

    # 2) build mask for x<x_max, y<y_max, z<z_max
    mask = (
        (cartesians[:,0] < x_max) &
        (cartesians[:,1] < y_max) &
        (cartesians[:,2] < z_max)
    )
    if not mask.any():
        return None

    # 3) pull species, and their Cartesian coords
    kept_cart = cartesians[mask]
    kept_spec = [struct[i].specie for i in np.nonzero(mask)[0]]

    # 4) create a new lattice of size (XMAX, YMAX, ZMAX)
    new_lat = Lattice.from_parameters(
        x_max, y_max, z_max,
        struct.lattice.alpha,
        struct.lattice.beta,
        struct.lattice.gamma
    )

    # 5) re‑fractionalize into the new cell
    #    f_new = cart / new_length  (componentwise)
    new_fracs = np.empty_like(kept_cart)
    new_fracs[:,0] = kept_cart[:,0] / x_max
    new_fracs[:,1] = kept_cart[:,1] / y_max
    new_fracs[:,2] = kept_cart[:,2] / z_max

    # 6) build the sub‑structure
    return Structure(new_lat, kept_spec, new_fracs, coords_are_cartesian=False)


if __name__ == "__main__":
    for fname in os.listdir("."):
        if not fname.lower().endswith(".cif"):
            continue

        print(f"Reading {fname} …")
        struct = Structure.from_file(fname)

        sub = extract_slice(struct, XMAX, YMAX, ZMAX)
        if sub is None:
            print("  → No atoms found in the 30×30×30 Å region; skipping.")
            continue

        out = f"sliced_a{int(XMAX)}_b{int(YMAX)}_c{int(ZMAX)}_{fname}"
        print(f"  → Writing {out}")
        sub.to(filename=out)


Reading ptmsp.cif …
  → Writing sliced_a30_b30_c30_ptmsp.cif
Reading PIM1.cif …
  → Writing sliced_a30_b30_c30_PIM1.cif
Reading 6fdadurene.cif …
  → Writing sliced_a30_b30_c30_6fdadurene.cif
